In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
from scipy.special import expit
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
# X, y = make_classification(n_samples=1000, n_features=10, n_classes=2, random_state=42)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train=pd.read_csv('X_train_classification.csv')
X_train=X_train.to_numpy()
X_test=pd.read_csv('X_test_classification.csv')
X_test=X_test.to_numpy()
y_train=pd.read_csv('y_train_classification.csv')
y_train=y_train.to_numpy()
y_train=y_train.reshape(-1)
y_test=pd.read_csv('y_test_classification.csv')
y_test=y_test.to_numpy().reshape(-1)

In [3]:
from phe import paillier
public_key,private_key=paillier.generate_paillier_keypair()

In [4]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [5]:
def paillier_multiplication_edge(a,b):
    a=private_key.decrypt(a)
    a=a*b
    return a

In [6]:
def paillier_multiplication_cloud(a,b):
    noise1=10
    noise2=20
    a=a+noise1
    sub_mass=(b*noise1)*(-1)
    sol=paillier_multiplication_edge(a,b)
    sol=sol+sub_mass
    return sol

In [7]:
def sigmoid(z):
    z=private_key.decrypt(z)
    z=expit(z)
    z=public_key.encrypt(z)
    return z

In [8]:
m, n = X_train.shape
weights = np.zeros(n)
bias = 0
learning_rate=0.01

In [9]:
enc_X_train=[]
for i in X_train:
    row_in=[]
    for j in i:
        p=public_key.encrypt(j)
        row_in.append(p)
    enc_X_train.append(row_in)

In [10]:
enc_y_train=[]
for i in y_train:
    p=public_key.encrypt(int(i))
    enc_y_train.append(p)

In [11]:
def logistic_regression_sgd(X, y, n, learning_rate=0.01, epochs=3):
    weights = np.zeros(n)
    
    weights_enc=[]
    for i in weights:
        p=public_key.encrypt(i)
        weights_enc.append(p)
        
    bias=0
    bias_enc=public_key.encrypt(bias)
    
    for epoch in range(epochs):
        indices = np.random.permutation(m)
        for i in indices:
            xi=enc_X_train[i]
            yi=enc_y_train[i]
            
            linear_output=0
            for i,j in zip(xi,weights_enc):
                p=paillier_multiplication_cloud(i,j)
                linear_output=linear_output+p
            
            linear_output=linear_output+bias_enc
            y_pred=sigmoid(linear_output)
            db=y_pred + (yi * (-1))
            dw=[]
            for i in xi:
                p=paillier_multiplication_cloud(i,db)
                dw.append(p)
            
            for i in range(len(weights_enc)):
                p=dw[i]*(-1)*learning_rate
                weights_enc[i]=weights_enc[i]+p
            
            bias_enc=bias_enc+(db*-1)*learning_rate
            
        

    return weights_enc, bias_enc

In [12]:
weights_enc, bias_enc = logistic_regression_sgd(enc_X_train,enc_y_train,n)

In [13]:
for i in weights_enc:
    print(private_key.decrypt(i))

0.12249626493318287
0.7054969660353599


In [14]:
private_key.decrypt(bias_enc)

0.0017726227355320853